# IBM Data Science Capstone Project

##### This notebook will be used to complete the IBM Data Science online courses' final project, related to geodata and the use of the location data-based Foursquare API

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

## Description of the Project

The aim of this project is to provide potential business starters with some orientation regarding where to locate their economic activities in the regions of Spain. Since both Barcelona and Madrid represent the largest contribution to the Gross Domestic Product (GDP) of Spain ([1](https://es.wikipedia.org/wiki/Anexo:Provincias_de_Espa%C3%B1a_por_PIB)), they are the most suited to establish a new business and thus, the focus of this project.

By scraping all of the municipalities from both cities and their contributions to the Spanish GDP, we will restrict the analysis to the ones that generate the most; then, an analysis of the local venues will be carried out to get a general idea of the type of businesses that are successful in the cities.
Finally, general conclusions and a recommendation will be drawn.

* **Required Data and its scraping**

First, we need to know the different areas that constitute both cities. Luckily for us, we can find this information in Wikipedia (([2](https://en.wikipedia.org/wiki/Municipalities_of_Catalonia)), ([3](https://en.wikipedia.org/wiki/List_of_municipalities_in_the_Community_of_Madrid))).
All of the necessary GDP-related data can also be found in ([4](https://datosmacro.expansion.com/mercado-laboral/renta/espana/municipios)).

The geographical coordinates of all of the municipalities will be scraped by the Geocoder Python package ([5](https://geocoder.readthedocs.io/index.html)), and the local venues will be obtained via the Foursquare API ([6](https://developer.foursquare.com/docs/places-api/)).

---

**Since Github is not able to show maps generated by the Folium package (which will be used for this project), it is highly advised to read this notebook under Nbviewer ([7](https://nbviewer.jupyter.org/)).**